In [1]:
#!pip install pyspellchecker
#!pip install plotly

In [2]:
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
from collections import Counter
from pathlib import Path
from unidecode import unidecode
from enchant.checker import SpellChecker
#from spellchecker import SpellChecker
import numbers
import sys

In [3]:
import string
from nltk.corpus import words
choices = words.words()
##Get vocabulary to be used for recognised words
with open('vocabulary.txt', 'r+', encoding='utf-8') as f:
    wordlist_tr_pre = f.readlines()

wordlist_tr = [x.strip().lower() for x in wordlist_tr_pre]

In [4]:
choices= set(wordlist_tr + words.words() + list(string.punctuation))
#choice_list with vocabulary used for unrecognised words ignorewords only
choices_list = wordlist_tr + words.words() + list(string.punctuation)

In [6]:
#get doc and page data frame
df_doc=pd.DataFrame()
all_text_content=""
for txt_path in Path("txt").glob("*.txt"):
    print(txt_path)
    doc_id=txt_path.stem
    text_content = Path(txt_path).read_text()
    all_text_content=all_text_content+" "+text_content
    splitted_pages = text_content.split("## p.")
    df_page = pd.DataFrame(splitted_pages, columns = ['text'])
    df_page["doc_id"]=doc_id
    df_page.reset_index(inplace=True)
    df_page = df_page.rename(columns = {'index':'page_id'})
    print(df_page.info())
    df_doc = pd.concat([df_doc, df_page])
print(df_doc.info())

txt/14.txt
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 3 columns):
page_id    545 non-null int64
text       545 non-null object
doc_id     545 non-null object
dtypes: int64(1), object(2)
memory usage: 12.9+ KB
None
txt/34.txt
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 3 columns):
page_id    477 non-null int64
text       477 non-null object
doc_id     477 non-null object
dtypes: int64(1), object(2)
memory usage: 11.3+ KB
None
txt/4.txt
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 3 columns):
page_id    419 non-null int64
text       419 non-null object
doc_id     419 non-null object
dtypes: int64(1), object(2)
memory usage: 9.9+ KB
None
txt/27.2.txt
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 3 columns):
page_id    413 non-null int64
text       413 non-null object
doc_id     413 non-null object
dtyp

In [7]:
#sorted by doc and page id
df_doc.sort_values(by=['doc_id','page_id'], inplace=True, ascending=[False, True])
print(df_doc.head())

   page_id                                               text doc_id
0        0  This file was downloaded from HathiTrust Digit...    9.4
1        1   (#1) ########################################...    9.4
2        2   (#2) ########################################...    9.4
3        3   (#3) ########################################...    9.4
4        4   (#4) ########################################...    9.4


In [8]:
#print(stopwords.words('english'))

en_stopwords = stopwords.words('english')
ex_list = ['the']
en_stopwords.extend(ex_list)

en_stopwords = en_stopwords + ["&","!", ",", ".", "\"", "?", '(', ')', '*', "'", '#', '1', '2','3','4','5','6','7','8','9']


print(en_stopwords)

ignorewords_1 = [":","&","!", ",", ".", "\"", "?", '(', ')', '*', "'", '#', '1', '2','3','4','5','6','7','8','9']
ignorewords = ignorewords_1 + choices_list
# use lang spellcheck for unrecognised words
d = SpellChecker()
d_en = SpellChecker('en') #en_GB
d_fr = SpellChecker('fr')
d_de = SpellChecker('de-de')

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [9]:


def get_spellcheck(text):
    if(d_en.check(text)):
        return True
    if(d_fr.check(text)):
        return True
    if(d_de.check(text)):
        return True
    return False
    

In [10]:

def check_isnumber(word):
    if word.isdigit():
        num = int(word)
    else:
        try:
            num = float(word)
        except:
            num = "string"
    return isinstance(num, numbers.Number)


In [11]:

def get_text_counter(text_content):
    text_content=unidecode(text_content)
    text_content = text_content.replace("\n", " ")
    text_content = text_content.replace("#", " ")
    #noted for sep number remove
    text_content = text_content.replace(".", " . ")
    # cleanup text
    rep = { '\n': ' ','\t': ' ', '\\': ' ', '\"': ' ', '-': ' ', '"': ' ', 
            ',':' ', '.':' ', '!':' ','.':' ',';':' ',"'":' ','<':' ',
            '?':' ', "n't": " not" , "'ll": " will", '*':' ','>':' ',
            '(': ' ', ')': ' ', "s'": "s ",':':' ','/':' ', '{}':' ', '{ }':' '}
    # remove words only containing number 
    text_content = re.sub(r'\b[0-9]+\b\s*','', text_content)
    
    rep = dict((re.escape(k), v) for k, v in rep.items()) 
    pattern = re.compile("|".join(rep.keys()))
    text_content = pattern.sub(lambda m: rep[re.escape(m.group(0))], text_content)
    voc_counter = Counter(word.lower() for word in text_content.rstrip().split() if not len(word)<2 if not check_isnumber(word) if not get_spellcheck(word) if not word.lower() in ignorewords )
    
    return voc_counter


def get_wordlist_text_counter(text_content):
    text_content = text_content.replace('":"', " ")
    text_content = text_content.replace('","', " ")
    text_content = text_content.replace('{"', " ")
    text_content = text_content.replace('"}', " ")
    text_content = text_content.replace("{}", " ")
    text_content = text_content.replace("{ }", " ")
    text_content = text_content.replace("\\", " ")
    text_content = text_content.replace("/", " ")
    text_content = text_content.replace("<", " ")
    text_content = text_content.replace(">", " ")
    # remove words only containing number 
    text_content = re.sub(r'\b[0-9]+\b\s*','', text_content)
    
    with open('test.txt', 'w') as f:
        f.write(text_content)                            
     
        
    voc_counter = Counter(word for word in text_content.rstrip().split() if not len(word)<2  )
    
    return voc_counter

In [12]:
#all vocabulary count
def get_all_count(text_content):
    
    all_voc_counter = get_text_counter(text_content)
    all_voc_count = len(all_voc_counter)
    print(all_voc_count)
    by_count = [(k, v) for k, v in all_voc_counter.items()]
    df_count = pd.DataFrame(by_count, columns = ['Word', 'Count'])
    df_count.sort_values(by=['Count'], inplace=True, ascending=False)
    print(df_count)
    df_count.to_csv('all_voc_count.csv')
    return all_voc_count

In [ ]:
all_voc_count = get_all_count(all_text_content)

In [ ]:
# function to count words
def get_df_by_word(page_voc_counter):
    by_word = [(k, v) for k, v in page_voc_counter.items()]
    df_by_word = pd.DataFrame(by_word, columns = ['Word', 'Count'])
    df_by_word.sort_values(by=['Count'], inplace=True, ascending=False)
    df_by_word=df_by_word.reset_index(drop=True)
    return df_by_word

In [ ]:
# sort df by page voc/word count
def get_sort_page_voc_count(df_page):
    # get each page vocabulary count, and sort data frame by the counts 
    voc_count = []
    word_list = []
    count_list = []

    for page_text in df_page["text"]:
        page_voc_counter = get_text_counter(page_text)
        df_by_word=get_df_by_word(page_voc_counter)
        page_word_list=df_by_word['Word'].to_json()
        page_count_list=df_by_word['Count'].to_json()
        page_voc_count = len(page_voc_counter)
        word_list.append(page_word_list)
        count_list.append(page_count_list)
        voc_count.append(page_voc_count)

    df_page["voc_count"] = voc_count   
    df_page["word_list"] = word_list  
    df_page["count_list"] = count_list  
    
    df_page.sort_values(by=['voc_count'], inplace=True, ascending=False)
    #print(df_page)
    return df_page


In [ ]:
df_doc = get_sort_page_voc_count(df_doc)
print(df_doc)

In [ ]:
import sys
# output files (txt/csv) for df with doc page voc/word count
def all_pages_txt(df_page):
    # selecting rows based on condition
    sel_pages = df_page #.sort_values('voc_count', ascending=False)
    #display(sel_pages[['page_id','voc_count','accum_voc_count', 'accum_voc_rate']])

    print(sel_pages.shape[0])
    #with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        #display(sel_pages[['page_id','voc_count','accum_voc_count', 'accum_voc_rate','word_list','count_list']])

    original_stdout = sys.stdout
    with open('M_ht_pages0.txt', 'w') as f:
        sys.stdout = f # Change the standard output to the file we created.
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
            print(sel_pages.shape[0])
            print(sel_pages[['doc_id','page_id','voc_count','word_list','count_list']])
        sys.stdout = original_stdout # Reset the standard output to its original value
    sel_pages[['doc_id','page_id','voc_count','word_list','count_list']].to_csv('M_ht_pages0.csv')

all_pages_txt(df_doc)

In [ ]:
# test wordlists.txt/test.txt
import sys
#sys.setrecursionlimit(1500)
original_stdout = sys.stdout
accum_text=""
with open('word_lists.txt', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        #print((sel_pages[['page_id']]).to_string(index=False))
        i=0
        for word_list in df_doc["word_list"]:  
            print(i)
            print("\n\n")
            print(word_list)
            accum_text = accum_text+ " " + word_list
            accum_text = accum_text.replace('":"', " ")
            accum_text = accum_text.replace('","', " ")
            accum_text = accum_text.replace('{"', " ")
            accum_text = accum_text.replace('"}', " ")
            accum_text = accum_text.replace("{}", " ")
            accum_text = accum_text.replace("{ }", " ")
            # remove words only containing number 
            accum_text = re.sub(r'\b[0-9]+\b\s*','', accum_text)
            print(accum_text)
            print("\n\n")
            print(i)
            i=i+1
            page_voc_counter = get_wordlist_text_counter(accum_text)
            df_by_word = get_df_by_word(page_voc_counter)
            updated_accum_text = df_by_word['Word'].to_json(default_handler=str)
            accum_text = updated_accum_text
            if(i>3):
                break
            
    sys.stdout = original_stdout # Reset the standard output to its original value

In [ ]:
# function to get page accumulated voc count
def get_sort_page_accum_voc_count(df_page):
    # get accumulated vocabulary counts from previous ordered pages
    accum_voc_count = []
    accum_voc_rate = []
    accum_text = ""
    accum_text_test = []
    #i=0
    for word_list in df_page["word_list"]:
        
        accum_text = accum_text+ " " + word_list  
        '''accum_text = accum_text.replace('":"', " ")
        accum_text = accum_text.replace('","', " ")
        accum_text = accum_text.replace('{"', " ")
        accum_text = accum_text.replace('"}', " ")'''
        
        #only using actual word count list from accum text to reduct text size
        page_voc_counter = get_wordlist_text_counter(accum_text)
        df_by_word = get_df_by_word(page_voc_counter)
        updated_accum_text = df_by_word['Word'].to_json(default_handler=str)
        accum_text = updated_accum_text
        
        accum_text_test.append(accum_text)
        page_voc_count = len(page_voc_counter)
        accum_voc_count.append(page_voc_count)
        accum_voc_rate.append(page_voc_count/all_voc_count)
        #i=i+1
        #if(i==230):
            #print(accum_text)

    df_page["accum_voc_count"] = accum_voc_count 
    df_page["accum_voc_rate"] = accum_voc_rate 

    #print(df_page)
    return df_page 

In [ ]:
df_doc = get_sort_page_accum_voc_count(df_doc)
print(df_doc)

In [ ]:
import sys
def select_pages_on_accum_1(df_page):
    # selecting rows based on condition
    sel_pages = df_page[df_page['accum_voc_rate'] <=0.70] #.sort_values('voc_count', ascending=False)
    #display(sel_pages[['page_id','voc_count','accum_voc_count', 'accum_voc_rate']])

    print(sel_pages.shape[0])
    #with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        #display(sel_pages[['page_id','voc_count','accum_voc_count', 'accum_voc_rate','word_list','count_list']])

    original_stdout = sys.stdout
    with open('M_ht_pages.txt', 'w') as f:
        sys.stdout = f # Change the standard output to the file we created.
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
            #print((sel_pages[['page_id']]).to_string(index=False))
            print(sel_pages.shape[0])
            print(sel_pages[['doc_id','page_id','voc_count','accum_voc_count', 'accum_voc_rate','word_list','count_list']])
        sys.stdout = original_stdout # Reset the standard output to its original value
    sel_pages[['doc_id','page_id','voc_count','accum_voc_count', 'accum_voc_rate','word_list','count_list']].to_csv('M_ht_pages.csv')

In [ ]:
select_pages_on_accum_1(df_doc)

In [ ]:
# function to get page incremented voc count
def get_sort_page_incre_voc_count(df_page):
    # get  accumulated vocabulary increment counts from previous ordered pages
    accum_voc_incre_count = []
    last_accum_voc_count = 0

    for accum_voc_count in df_page["accum_voc_count"]:
        current_accum_voc_incre_count = accum_voc_count-last_accum_voc_count
        accum_voc_incre_count.append(current_accum_voc_incre_count)
        last_accum_voc_count=accum_voc_count

    df_page["accum_voc_incre_count"] = accum_voc_incre_count 

    # sort by increment count first then vocabulary count
    df_page.sort_values(by=['accum_voc_incre_count','voc_count'], inplace=True, ascending=[False, False])

    #print(df_page)
    return df_page

In [ ]:
df_doc=get_sort_page_incre_voc_count(df_doc)
print(df_doc)

In [ ]:
# function to get page accumulated voc count 2 with increment step
def get_sort_page_accum2_voc_count(df_page):
    #redo the accum counts and rates
    # get accumulated vocabulary counts from previous ordered pages
    accum_voc_count2 = []
    accum_voc_rate2 = []
    accum_text = ""

    for word_list in df_page["word_list"]:
        
        accum_text = accum_text+ " " + word_list  
        '''accum_text = accum_text.replace('":"', " ")
        accum_text = accum_text.replace('","', " ")
        accum_text = accum_text.replace('{"', " ")
        accum_text = accum_text.replace('"}', " ")'''
        
        #only using actual word count list from accum text to reduct text size
        page_voc_counter = get_wordlist_text_counter(accum_text)
        df_by_word = get_df_by_word(page_voc_counter)
        updated_accum_text = df_by_word['Word'].to_json()
        accum_text = updated_accum_text

        page_voc_count = len(page_voc_counter)
        accum_voc_count2.append(page_voc_count)
        accum_voc_rate2.append(page_voc_count/all_voc_count)

    df_page["accum_voc_count2"] = accum_voc_count2 
    df_page["accum_voc_rate2"] = accum_voc_rate2 

    #print(df_page)
    return df_page

In [ ]:
df_doc=get_sort_page_accum2_voc_count(df_doc)
print(df_doc)

In [ ]:
# output results on different conditions
def select_pages_on_accum_2(df_page):
    # selecting rows based on condition
    sel_pages2 = df_page[df_page['accum_voc_rate2'] <=0.70]
    #display(sel_pages[['page_id','voc_count','accum_voc_count', 'accum_voc_rate']])

    print(sel_pages2.shape[0])
    #with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        #display(sel_pages2[['page_id','voc_count','accum_voc_count2', 'accum_voc_rate2','word_list','count_list']])
        
    original_stdout = sys.stdout
    with open('M_ht_pages_2.txt', 'w') as f:
        sys.stdout = f # Change the standard output to the file we created.
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
            #print((sel_pages[['page_id']]).to_string(index=False))
            print(sel_pages2.shape[0])
            print(sel_pages2[['page_id','doc_id','voc_count','accum_voc_incre_count','accum_voc_count2', 'accum_voc_rate2','word_list','count_list']])
        sys.stdout = original_stdout # Reset the standard output to its original value
    sel_pages2[['page_id','doc_id','voc_count','accum_voc_incre_count','accum_voc_count2', 'accum_voc_rate2','word_list','count_list']].to_csv('M_ht_pages_2.csv')


In [ ]:
select_pages_on_accum_2(df_doc)

In [ ]:
df_doc[['page_id','doc_id','voc_count','accum_voc_incre_count','accum_voc_count2', 'accum_voc_rate2','word_list','count_list']].to_csv('M_ht_pages_3.csv')